In [1]:
# lib manipulasi data
import pandas as pd
import numpy as np

# Pustaka untuk visualisasi acf dan pacf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf

# pustaka uji stasioneritas arch
from arch.unitroot import *
from arch.unitroot import ADF
from arch.unitroot import PhillipsPerron
from arch.unitroot import KPSS


In [2]:
# load csv
dataset = pd.read_csv("../dataset/BTC-USD-Norm.csv", parse_dates=["Date"])

In [3]:
# show metadata
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3408 entries, 0 to 3407
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    3408 non-null   datetime64[ns]
 1   Open    3408 non-null   float64       
 2   High    3408 non-null   float64       
 3   Low     3408 non-null   float64       
 4   Close   3408 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 133.3 KB
None


In [4]:
adf = ADF(y=dataset["Close"], lags=60, trend="ct")
print(adf.summary().as_text())

   Augmented Dickey-Fuller Results   
Test Statistic                 -2.783
P-value                         0.203
Lags                               60
-------------------------------------

Trend: Constant and Linear Time Trend
Critical Values: -3.96 (1%), -3.41 (5%), -3.13 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


In [5]:
pp = PhillipsPerron(y=dataset["Close"], lags=60, trend="ct")
print(pp.summary().as_text())

     Phillips-Perron Test (Z-tau)    
Test Statistic                 -2.371
P-value                         0.395
Lags                               60
-------------------------------------

Trend: Constant and Linear Time Trend
Critical Values: -3.96 (1%), -3.41 (5%), -3.13 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


In [6]:
kpss = KPSS(y=dataset["Close"], lags=60, trend="ct")
print(kpss.summary().as_text())

    KPSS Stationarity Test Results   
Test Statistic                  0.241
P-value                         0.006
Lags                               60
-------------------------------------

Trend: Constant and Linear Time Trend
Critical Values: 0.22 (1%), 0.15 (5%), 0.12 (10%)
Null Hypothesis: The process is weakly stationary.
Alternative Hypothesis: The process contains a unit root.
